




```
source /pyenv/bin/activate
sudo mkdir -p /workdir
sudo chmod -R 777 /workdir
cd /workdir
pip install jupyter
nohup jupyter notebook --ip 0.0.0.0 --port 8888 &
```

```
sudo apt install unzip
pip install cassandra-driver
curl https://certs.secureserver.net/repository/sf-class2-root.crt -O
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install
aws configure
```

In [2]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2 , CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider

In [3]:
ssl_context = SSLContext(PROTOCOL_TLSv1_2 )
ssl_context.load_verify_locations('sf-class2-root.crt')

/tmp/ipykernel_137903/972152945.py:1: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2 )


In [4]:
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username='u02', password='')

In [6]:
cluster = Cluster(['cassandra.us-east-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)

In [7]:
session = cluster.connect()

NoHostAvailable: ('Unable to connect to any servers', {'3.234.248.234:9142': AuthenticationFailed('Failed to authenticate to 3.234.248.234:9142: Error from server: code=0100 [Bad credentials] message="Provided username u02 and/or password are incorrect"')})

In [8]:
r = session.execute('select * from system_schema.keyspaces')
print(r.current_rows)

NameError: name 'session' is not defined

In [9]:
rows = session.execute("SELECT * FROM ag_aws_ks.employees")
print(rows)

NameError: name 'session' is not defined

In [10]:
for row in rows:
    print(row)

NameError: name 'rows' is not defined

In [12]:
from cassandra.cluster import Cluster

cluster2 = Cluster(['localhost'], port=9042)
session2 = cluster2.connect()


In [18]:
command = """
CREATE KEYSPACE IF NOT EXISTS target

WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 2};
"""

In [19]:
session2.execute(command)

In [20]:
command = """
CREATE TABLE IF NOT EXISTS target.employees (
   no int PRIMARY KEY,
   name text)
"""

In [21]:
session2.execute(command)

In [22]:
rows = session.execute("SELECT * FROM ag_aws_ks.employees")
print(rows)

In [23]:
for row in rows:
    print(row[0], row[1])
    session2.execute(f"insert into target.employees(no, name) values ({row[0]}, '{row[1]}')")

1 Atin


In [24]:
rows2 = session2.execute("SELECT * FROM target.employees")

for row2 in rows2:
    print(row2)

Row(no=1, name='Atin')


In [25]:
! pip install amazon-dax-client
! pip install boto3

In [26]:
import boto3

In [27]:
dyn_resource = boto3.resource('dynamodb', region_name="us-east-1")

In [28]:
table_name = 'TryDaxTable'
params = {
    'TableName': table_name,
    'KeySchema': [
        {'AttributeName': 'partition_key', 'KeyType': 'HASH'},
        {'AttributeName': 'sort_key', 'KeyType': 'RANGE'}
    ],
    'AttributeDefinitions': [
        {'AttributeName': 'partition_key', 'AttributeType': 'N'},
        {'AttributeName': 'sort_key', 'AttributeType': 'N'}
    ],
    'ProvisionedThroughput': {
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 10
    }
}

In [60]:
try:
    table = dyn_resource.create_table(**params)
    print(f"Creating {table_name}...")
    table.wait_until_exists()
except:
    print("Error")

Error


In [62]:
table = dyn_resource.Table(table_name)

In [63]:
key_count = 10
item_size = 10

some_data = 'X' * item_size
for partition_key in range(1, key_count + 1):
    for sort_key in range(1, key_count + 1):
        table.put_item(Item={
            'partition_key': partition_key,
            'sort_key': sort_key,
            'some_data': some_data
        })
        print(f"Put item ({partition_key}, {sort_key}) succeeded.")

Put item (1, 1) succeeded.
Put item (1, 2) succeeded.
Put item (1, 3) succeeded.
Put item (1, 4) succeeded.
Put item (1, 5) succeeded.
Put item (1, 6) succeeded.
Put item (1, 7) succeeded.
Put item (1, 8) succeeded.
Put item (1, 9) succeeded.
Put item (1, 10) succeeded.
Put item (2, 1) succeeded.
Put item (2, 2) succeeded.
Put item (2, 3) succeeded.
Put item (2, 4) succeeded.
Put item (2, 5) succeeded.
Put item (2, 6) succeeded.
Put item (2, 7) succeeded.
Put item (2, 8) succeeded.
Put item (2, 9) succeeded.
Put item (2, 10) succeeded.
Put item (3, 1) succeeded.
Put item (3, 2) succeeded.
Put item (3, 3) succeeded.
Put item (3, 4) succeeded.
Put item (3, 5) succeeded.
Put item (3, 6) succeeded.
Put item (3, 7) succeeded.
Put item (3, 8) succeeded.
Put item (3, 9) succeeded.
Put item (3, 10) succeeded.
Put item (4, 1) succeeded.
Put item (4, 2) succeeded.
Put item (4, 3) succeeded.
Put item (4, 4) succeeded.
Put item (4, 5) succeeded.
Put item (4, 6) succeeded.
Put item (4, 7) succeeded

In [64]:
from boto3.dynamodb.conditions import Key

In [65]:
partition_key = 5
sort_keys = (2, 9)

In [66]:
key_condition_expression = Key('partition_key').eq(partition_key) & Key('sort_key').between(*sort_keys)

In [67]:
out = table.query(KeyConditionExpression=key_condition_expression)

In [68]:
for r in out['Items']:
    print(r)

{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('2'), 'partition_key': Decimal('5')}
{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('3'), 'partition_key': Decimal('5')}
{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('4'), 'partition_key': Decimal('5')}
{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('5'), 'partition_key': Decimal('5')}
{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('6'), 'partition_key': Decimal('5')}
{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('7'), 'partition_key': Decimal('5')}
{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('8'), 'partition_key': Decimal('5')}
{'some_data': 'XXXXXXXXXX', 'sort_key': Decimal('9'), 'partition_key': Decimal('5')}
